In [2]:
import numpy as np
from music21 import instrument, note, chord, stream
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
from tensorflow.keras.utils import to_categorical
import random

# STEP 1: Create Synthetic Notes Dataset
def generate_synthetic_notes():
    pitches = ['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4']
    notes = [random.choice(pitches) for _ in range(500)]
    return notes

notes = generate_synthetic_notes()

# STEP 2: Prepare Sequences
sequence_length = 10
pitchnames = sorted(set(notes))
note_to_int = {note: number for number, note in enumerate(pitchnames)}

network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i + sequence_length]
    seq_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in seq_in])
    network_output.append(note_to_int[seq_out])

n_patterns = len(network_input)
n_vocab = len(pitchnames)

network_input = np.reshape(network_input, (n_patterns, sequence_length, 1)) / float(n_vocab)
network_output = to_categorical(network_output)

# STEP 3: Build the Model
model = Sequential()
model.add(LSTM(128, input_shape=(network_input.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dense(64))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# STEP 4: Train the Model
model.fit(network_input, network_output, epochs=30, batch_size=32)

# STEP 5: Generate Notes
int_to_note = {number: note for note, number in note_to_int.items()}

def generate_notes(model, network_input, n_vocab):
    start = np.random.randint(0, len(network_input) - 1)
    pattern = network_input[start].reshape(1, sequence_length, 1)
    prediction_output = []

    for _ in range(50):
        prediction = model.predict(pattern, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern = np.append(pattern[:, 1:, :], [[[index / float(n_vocab)]]], axis=1)

    return prediction_output

# STEP 6: Convert Notes to MIDI
def create_midi(prediction_output, filename="simple_output.mid"):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=filename)

# Generate and Save Music
prediction_output = generate_notes(model, network_input, n_vocab)
create_midi(prediction_output)


C:\Users\Junaid Ahamed\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - loss: 1.9510
Epoch 2/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 1.9473
Epoch 3/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 1.9424
Epoch 4/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 1.9409
Epoch 5/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 1.9386
Epoch 6/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.9433
Epoch 7/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.9401
Epoch 8/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 1.9397
Epoch 9/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 1.9458
Epoch 10/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 1.9423
Epoch 11/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 1.9420
Epoch 12/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.9374
Epoch 13/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 1.9445
Epoch 14/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1.9385
Epoch 15/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 1.9384
Epoc